In [10]:
import pandas as pd
import numpy as np
import re
from glob import glob
from tqdm import tqdm
import optuna
# import optuna.integration.lightgbm as lgb
import lightgbm as lgb

from lightgbm import early_stopping
from lightgbm import log_evaluation

from sklearn.model_selection import train_test_split
from wandb.lightgbm import wandb_callback
import wandb

from preprocessing import preprocessing

from tqdm.auto import tqdm
tqdm.pandas()

wandb.init(project="narou", entity="ryotoitoi", name = "exp_6_narou")

### ファイル読み込み・データ確認

df_train = pd.read_pickle('./data/train.pkl')
df_test = pd.read_pickle('./data/test.pkl')
sub_df = pd.read_csv('../data/sample_submission.csv')
train_x = df_train.drop(columns="fav_novel_cnt_bin")
train_y = df_train[["fav_novel_cnt_bin"]]

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.2)
params = {
    'objective': 'multiclass',
    'num_classes': 5,
    "verbosity": -1,
    'metric': 'multi_logloss',
    "seed": 42
}

train_data = lgb.Dataset(train_x, label=train_y)
val_data = lgb.Dataset(val_x, label=val_y)

cat_cols = ["ncode_num", "userid", 'biggenre', 'genre', 'novel_type', 'end', 'isstop', 'isr15', 'isbl', 'isgl', 'iszankoku', 'istensei', 'istenni', 'pc_or_k']

print("start training")
model = lgb.train(
    params,
    train_data, 
    categorical_feature = cat_cols,
    valid_names = ['train', 'valid'],
    valid_sets =[train_data, val_data], 
    callbacks=[wandb_callback(), early_stopping(30), log_evaluation(30)], 
)

train_multi_logloss,█▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
valid_multi_logloss,█▆▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_multi_logloss,0.3277
valid_multi_logloss,0.79775


start training


/usr/local/lib/python3.8/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['biggenre', 'end', 'genre', 'isbl', 'isgl', 'isr15', 'isstop', 'istenni', 'istensei', 'iszankoku', 'ncode_num', 'novel_type', 'pc_or_k', 'userid']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/usr/local/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Training until validation scores don't improve for 30 rounds
[30]	train's multi_logloss: 0.568122	valid's multi_logloss: 0.820637
[60]	train's multi_logloss: 0.417903	valid's multi_logloss: 0.808743
Early stopping, best iteration is:
[55]	train's multi_logloss: 0.437198	valid's multi_logloss: 0.808255


In [12]:
test_pred = model.predict(df_test, num_iteration=model.best_iteration)
sub_df.iloc[:, 1:] = test_pred
sub_df.to_csv('./output/exp6_submission.csv', index=False)